In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pysqkit
from pysqkit.util.linalg import get_mat_elem
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import json

In [ ]:
with open("../../flx_transm_params.txt") as my_file:
    parameters_set = json.load(my_file)

In [ ]:
# Comment/Uncomment p_set to select the parameter set
p_set = "CR_3"
#p_set = "CPHASE"

#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set]["max_freq_t"], 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=None, 
    env_thermal_energy=None,    
    dim_hilbert=levels_t,
    dephasing_times=None
)

#Fluxonium
levels_f = 6

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], 
    env_thermal_energy=None,
    dephasing_times=None
)
flx.diagonalize_basis(levels_f)

flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cr_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

jc = parameters_set[p_set]["jc"]
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
bare_system = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0.0)

states_label = coupled_sys.all_state_labels()
states_dict = coupled_sys.states_as_dict(as_qobj=True)

In [ ]:
levels_to_plot = ['00', '01', '10', '11', '02', '20', '12','21', '03', '13', '04']

ground_energy = bare_system.state('00')[0]
energy = {}
state = {}
x_avg = {}

for level in levels_to_plot:
    energy[level] = bare_system.state(level)[0] - ground_energy
    state[level] = bare_system.state(level)[1]

qubit_labels = bare_system.labels
n1n2_op = bare_system[qubit_labels[0]].charge_op().dot(bare_system[qubit_labels[1]].charge_op())
abs_mat_elem_list = []
count = 0
for level_1 in levels_to_plot:
    for level_2 in levels_to_plot:
        abs_mat_elem_list.append(np.abs(get_mat_elem(n1n2_op, state[level_1], state[level_2])))
max_abs_mat_elem = max(abs_mat_elem_list)

abs_mat_elem = {}

for level_1 in levels_to_plot:
        for level_2 in levels_to_plot:
            abs_mat_elem[level_1 + "_" + level_2] = np.abs(get_mat_elem(n1n2_op, state[level_1], state[level_2]))

my_data = {}
my_data["energy"] = energy
my_data["abs_mat_elem"] = abs_mat_elem

In [5]:
save = True
if save:
    with open('data_' + p_set + '.txt', 'w') as fp:
        json.dump(my_data, fp)